# IMPORTANDO AS BIBLIOTECAS

In [1]:
import datetime
import math
from os import path, listdir
from pathlib import Path
from inspect import stack

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from geopy.geocoders import Nominatim

# DEFININDO OPÇÕES GLOBAIS

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
plt.figure(figsize=(12, 8))

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# FUNÇÕES ÚTEIS

In [5]:
def get_files_directory(path_dir, specific_type=None):

    """

        FUNÇÃO PARA OBTER ARQUIVOS DE UM DIRETÓRIO.

        É POSSÍVEL ENVIAR UM FORMATO ESPECÍFICO PARA
        FILTRO DO FORMATO DE ARQUIVO DESEJADO.
        EX: OBTER APENAS XLSX

        # Arguments
            path_dir                   - Required : Diretório analisado (String)
            specific_type              - Optional : Lista com os formatos desejados (List)

        # Returns
            list_files                 - Required : Arquivos do diretório (List)

    """

    # INICIANDO A VARIÁVEL QUE ARMAZENARÁ TODOS OS ARQUIVOS DO DIRETÓRIO
    list_files = []

    # OBTENDO TODOS OS ARQUIVOS
    try:

        # VERIFICANDO SE É DIRETÓRIO
        if path.isdir(path_dir):

            # OBTENDO TODOS OS ARQUIVOS EXISTENTES NO ARQUIVO
            list_files = [path.join(path_dir, name) for name in listdir(path_dir)]

            # VERIFICANDO SE UMA EXTENSÃO ESPECÍFICA FOI DEFINIDA
            if specific_type:

              # VERIFICANDO SE A EXTENSÃO É DO TIPO TUPLE/LIST
              if not isinstance(specific_type, (tuple, list)):
                  specific_type = [specific_type]

              print("FILTRANDO PARA AS EXTENSÕES: {}".format(specific_type))

              # FILTRANDO OS VALORES DA LISTA
              list_files = [arq for arq in list_files if (Path(arq).suffix in (specific_type) or str(Path(arq).suffix).replace(".", "") in (specific_type))]

        else:
            list_files = [path_dir]

    except Exception as ex:
        print("ERRO NA FUNÇÃO: {} - {}".format(stack()[0][3], ex))

    return list_files

In [16]:
geocoder = Nominatim(user_agent = 'describe_ghcn_daily')

def get_informations_using_lat_long(lat, long):

  """

        FUNÇÃO PARA OBTER AS INFORMAÇÕES
        DE UMA LOCALIZAÇÃO
        BASEADA EM SUA LATITUDE E LONGITUDE

        # Arguments
            lat                   - Required : Latitude do local (Int | Float)
            long                  - Optional : Longitude do local (Int | Float)

        # Returns
            location.raw          - Required : Informações do local (Dict)

    """

  try:
    # VERIFICANDO SE OS TIPOS DE DADOS NÃO SÃO NULOS
    if not math.isnan(lat) and not math.isnan(long):

      # VERIFICANDO SE OS TIPOS DE DADOS DE LAT E LONG SÃO FLOAT OU INT
      if isinstance(lat, (float, int)) and isinstance(long, (float, int)):

        # OBTENDO A LOCALIZAÇÃO
        location = geocoder.reverse((lat, long))

        # RETORNANDO O RESULTAOD OBTIDO
        return location.raw

  except Exception as ex:
        print("ERRO NA FUNÇÃO: {} - {}".format(stack()[0][3], ex))

  return None

# VARIÁVEIS GLOBAIS

In [17]:
# DADOS DE ESTAÇÕES CLIMÁTICAS
dir_stations = 'data/DATA_STATIONS/data_station.csv'

# OBTENDO OS DADOS DE ESTAÇÕES METEOLÓGICAS

In [8]:
# INICIANDO O DATAFRAME
df_station = pd.read_csv(dir_stations, sep=",")

In [9]:
df_station.head()

,name,country,state,lat,lon,elev,begin_station,end_station
0,IGARAPE-ACU NORTH,BR,NaN,-1.067,-47.617,57.0,19450514,19450920
1,BOCAIUVA SOUTHEAST,BR,NaN,-17.200,-43.667,828.0,19470406,19470523
2,OIAPOQUE,BR,NaN,3.814,-51.863,20.5,20160704,20190701
3,BOA VISTA,BR,NaN,2.805,-60.647,95.5,20160704,20210120
4,TARTARUGALZINHO,BR,NaN,1.497,-50.917,22.5,20161021,20180405


In [10]:
df_station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           941 non-null    object 
 1   country        941 non-null    object 
 2   state          0 non-null      float64
 3   lat            937 non-null    float64
 4   lon            937 non-null    float64
 5   elev           937 non-null    float64
 6   begin_station  941 non-null    int64  
 7   end_station    941 non-null    int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 58.9+ KB


In [11]:
df_station.describe()

,state,lat,lon,elev,begin_station,end_station
count,0.0,937.000000,937.000000,937.000000,9.410000e+02,9.410000e+02
mean,NaN,-15.457863,-47.819308,378.277695,1.992879e+07,2.015986e+07
std,NaN,8.473399,7.393010,362.592961,2.914832e+05,1.403735e+05
min,NaN,-33.750000,-72.787000,-999.900000,1.931010e+07,1.931093e+07
25%,NaN,-22.333000,-52.233000,79.000000,1.974061e+07,2.020030e+07
50%,NaN,-15.900000,-47.617000,302.500000,2.010111e+07,2.021092e+07
75%,NaN,-8.283000,-42.317000,617.000000,2.016070e+07,2.021092e+07
max,NaN,3.867000,-29.317000,1811.500000,2.021082e+07,2.021092e+07


# PRÉPROCESSAMENTO

## OBTENDO A COLUNA DE ANO DA ESTAÇÃO METEOLÓGICA

In [12]:
for column_name in ["begin_station", "end_station"]:

  print("APLICANDO A OBTENÇÃO DO ANO PARA A COLUNA: {}".format(column_name))

  df_station["{}_{}".format(column_name, "year")] = df_station[column_name].apply(lambda x: datetime.datetime.strptime(str(x), "%Y%m%d").year)

APLICANDO A OBTENÇÃO DO ANO PARA A COLUNA: begin_station
APLICANDO A OBTENÇÃO DO ANO PARA A COLUNA: end_station


# OBTENDO MAIS INFORMAÇÕES DA LOCALIZAÇÃO DAS ESTAÇÕES METEOLÓGICAS

In [18]:
# API GEOLOCALIZAÇÃO
df_station["information"] = df_station.apply(lambda x: get_informations_using_lat_long(lat=x["lat"],
                                                                                       long=x["lon"]), axis=1)

ERRO NA FUNÇÃO: get_informations_using_lat_long - 'NoneType' object has no attribute 'raw'


ERRO NA FUNÇÃO: get_informations_using_lat_long - 'NoneType' object has no attribute 'raw'


In [30]:
# OBTENDO CIDADE
df_station["city"] = df_station["information"].apply(lambda x: x.get("address").get("town") if isinstance(x, dict) else None)

In [33]:
# OBTENDO ESTADO
df_station["state"] = df_station["information"].apply(lambda x: x.get("address").get("state") if isinstance(x, dict) else None)

In [35]:
df_station

,name,country,state,lat,lon,elev,begin_station,end_station,begin_station_year,end_station_year,information,city
0,IGARAPE-ACU NORTH,BR,Pará,-1.067,-47.617,57.0,19450514,19450920,1945,1945,"{'place_id': 308598524, 'licence': 'Data © Ope...",Igarapé-Açu
1,BOCAIUVA SOUTHEAST,BR,Minas Gerais,-17.200,-43.667,828.0,19470406,19470523,1947,1947,"{'place_id': 285352067, 'licence': 'Data © Ope...",Bocaiuva
2,OIAPOQUE,BR,Amapá,3.814,-51.863,20.5,20160704,20190701,2016,2019,"{'place_id': 169784871, 'licence': 'Data © Ope...",Oiapoque
3,BOA VISTA,BR,Roraima,2.805,-60.647,95.5,20160704,20210120,2016,2021,"{'place_id': 45147859, 'licence': 'Data © Open...",Cantá
4,TARTARUGALZINHO,BR,Amapá,1.497,-50.917,22.5,20161021,20180405,2016,2018,"{'place_id': 257335651, 'licence': 'Data © Ope...",Tartarugalzinho
...,...,...,...,...,...,...,...,...,...,...,...,...
936,MOSTARDAS,BR,Rio Grande do Sul,-31.250,-50.900,5.0,19790112,20210920,1979,2021,"{'place_id': 202679614, 'licence': 'Data © Ope...",None
937,JOAO PESSOA,BR,Paraíba,-7.100,-34.867,7.0,19310104,20210920,1931,2021,"{'place_id': 122127204, 'licence': 'Data © Ope...",None
938,SANTA MARIA,BR,Sergipe,-10.984,-37.070,7.0,19830401,20210920,1983,2021,"{'place_id': 276760626, 'licence': 'Data © Ope...",None
939,BELEM,BR,Pará,-1.450,-48.467,10.0,19310104,20210920,1931,2021,"{'place_id': 43501370, 'licence': 'Data © Open...",None


## ORDENANDO O DATASET PELA ORDEM DAS ESTAÇÕES DO ANO

In [36]:
df_station = df_station.sort_values(by=["name", "begin_station", "end_station"], ascending=True)

# EXPORTANDO O RESULTADO

In [39]:
df_station.to_excel("data_station_with_result_api.xlsx", index=None)